In [1]:
import symmetric_svd_word_embedding_utilities as util

# Parameters

In [2]:
# whether consider the order of pmi
pmi_order = False

# threshold of rare unigram
rare_term = 10

# threshold of rare bigram
rare_bigram = 5

# dimension of matrix generated
dimension = 100

# value applied in laplace smooth
la_smooth = 1

# number of tweets deal with
num_of_tweets = 1000

# Import data

In [3]:
mypath = './train_tweets'
header_row = ['text','label']

In [4]:
df_raw = util.read_from_txt(mypath)

the filenames are ***************************
['tweets_senEval.txt']
Original size of data is  49828
after remove duplicate, the size is  49826




In [5]:
# mypath = '../tweets_set_ire'
# header_row = ['ID', 'ID_STR', 'author', 'text', 'time', 'location', 'latitude', 'longitude', 'screen_place']

In [6]:
# df_raw = util.read_from_csv(mypath,header_row)

In [7]:
print('the first row is*******************************')
print(df_raw.iloc[0])
print("**********************************************")

the first row is*******************************
text     Won the match #getin . Plus tomorrow is a very...
label                                              neutral
Name: 0, dtype: object
**********************************************


# Preprocessing tweet, remove mention, urls, emoji, strip each token and so on

In [8]:
# tweet_raw = util.preprocess_tweet(df_raw['text'][:num_of_tweets])
tweet_raw_ = util.preprocess_tweet(df_raw['text'])

In [9]:
tweet_raw=[]
for idx,row in df_raw.iterrows():
    if row['label'] == 'neutral':
        tweet_raw.append(tweet_raw_[idx]+['neutral_corpus'])
    elif row['label'] == 'positive':
        tweet_raw.append(tweet_raw_[idx]+['positive_corpus'])
    else:
        tweet_raw.append(tweet_raw_[idx]+['negative_corpus'])

In [10]:
print('the number of tweets are ****************************************',len(tweet_raw))
print("**********************************************")

the number of tweets are **************************************** 49826
**********************************************


In [11]:
print('first 5 tweets after preprocessing********************************')
print(tweet_raw[:5])
print("**********************************************")

first 5 tweets after preprocessing********************************
[['won', 'the', 'match', 'getin', '', 'plus', 'tomorrow', 'is', 'a', 'very', 'busy', 'day', 'with', 'awareness', "day's", 'and', 'debates', 'gulp', 'debates', 'neutral_corpus'], ['some', 'areas', 'of', 'new', 'england', 'could', 'see', 'the', 'first', 'flakes', 'of', 'the', 'season', 'tuesday', 'neutral_corpus'], ['2nd', 'worst', 'qb', 'definitely', 'tony', 'romo', 'the', 'man', 'who', 'likes', 'to', 'share', 'the', 'ball', 'with', 'everyone', 'including', 'the', 'other', 'team', 'negative_corpus'], ['thailand', 'washington', '', 'us', 'president', 'barack', 'obama', 'vowed', 'wednesday', 'as', 'he', 'visited', 'storm-ravaged', 'new', 'jersey', 'shore', 'to', 'neutral_corpus'], ['did', "y'all", 'hear', 'what', 'tony', 'romo', 'dressed', 'up', 'as', 'for', 'halloween', 'a', 'giants', 'quaterback', 'cause', "that's", 'all', 'he', 'could', 'throw', 'to', 'sunday', 'night', 'neutral_corpus']]
*******************************

# Compute the frequency of each term, remove the rare terms and stop words

In [12]:
punctuation = list(util.string.punctuation)
stop = util.stopwords.words('english') + punctuation + ['the','rt','&amp','a','','amp']
print('stop words are****************************************************')
print(stop)
print("**********************************************")

stop words are****************************************************
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
## count all terms frequency before removing stop words etc.
count_before_remove = util.word_counter(tweet_raw)

In [14]:
print('top 20 frequency words before removing*****************************')
print(count_before_remove.most_common(20))
print("**********************************************")
print("size of vocabulary before removing is ******************************",len(count_before_remove))
print("**********************************************")

top 20 frequency words before removing*****************************
[('the', 41256), ('to', 22876), ('neutral_corpus', 22333), ('positive_corpus', 19726), ('', 18639), ('in', 14827), ('on', 14264), ('a', 14081), ('i', 13869), ('and', 13777), ('of', 11858), ('for', 10831), ('is', 10606), ('you', 8904), ('with', 8258), ('be', 7994), ('tomorrow', 7792), ('negative_corpus', 7767), ('at', 7647), ('it', 7392)]
**********************************************
size of vocabulary before removing is ****************************** 53355
**********************************************


In [15]:
tweet_final = util.remove_rare_stop_words(tweet_raw,count_before_remove,rare_term,stop)

In [16]:
print('first 5 tweets after removing rare term and stop words**************')
print(tweet_final[:5])
print("**********************************************")

first 5 tweets after removing rare term and stop words**************
[['match', 'plus', 'tomorrow', 'busy', 'day', 'awareness', "day's", 'neutral_corpus'], ['areas', 'new', 'england', 'could', 'see', 'first', 'season', 'tuesday', 'neutral_corpus'], ['2nd', 'worst', 'qb', 'definitely', 'tony', 'romo', 'man', 'likes', 'share', 'ball', 'everyone', 'including', 'team', 'negative_corpus'], ['washington', 'us', 'president', 'barack', 'obama', 'wednesday', 'visited', 'new', 'jersey', 'shore', 'neutral_corpus'], ["y'all", 'hear', 'tony', 'romo', 'dressed', 'halloween', 'giants', 'cause', "that's", 'could', 'throw', 'sunday', 'night', 'neutral_corpus']]
**********************************************


In [17]:
# count all terms frequency after removing stop words etc.
counter_uni = util.word_counter(tweet_final)

In [18]:
print('top 20 frequency words after removing*******************************')
print(counter_uni.most_common(20))
print("**********************************************")
print("size of vocabulary after removing is",len(counter_uni))

top 20 frequency words after removing*******************************
[('neutral_corpus', 22333), ('positive_corpus', 19726), ('tomorrow', 7792), ('negative_corpus', 7767), ('may', 7354), ('day', 4182), ('going', 3293), ('night', 3134), ('see', 3081), ("i'm", 3035), ('friday', 2949), ('1st', 2845), ('sunday', 2761), ('like', 2722), ('time', 2576), ('get', 2464), ("it's", 2446), ('saturday', 2218), ('go', 2125), ('one', 2123)]
**********************************************
size of vocabulary after removing is 6034


In [19]:
tweet_final = util.remove_high_frequency_words(tweet_final,counter_uni,['positive_corpus','negative_corpus','neutral_corpus'],(1e-3)/5)

['ever', 'justin', 'back', 'game', 'way', 'wednesday', 'jason', 'day', 'national', 'new', 'right', 'going', 'thursday', 'could', 'paul', 'august', 'gonna', 'night', 'video', 'play', 'win', 'sun', '2nd', 'well', 'think', 'since', 'super', 'work', 'know', 'still', 'may', 'set', 'morning', 'world', 'sox', "i've", 'brother', 'want', 'sunday', 'next', 'saturday', 'even', '4th', '2', 'good', 'got', 'say', 'go', 'time', 'nicki', "you're", '10th', 'like', 'life', 'vs', 'need', 'west', '10', 'coming', 'monday', 'watch', 'every', 'september', 'another', 'god', 'march', 'milan', 'first', "i'm", 'cena', 'sat', 'red', 'sharknado', 'man', 'best', 'friday', 'two', 'tomorrow', 'see', 'would', '5', 'already', 'read', 'us', 'weekend', 'season', '1st', 'star', 'getting']


In [20]:
counter_uni = util.word_counter(tweet_final)

In [21]:
print('top 20 frequency words after removing*******************************')
print(counter_uni.most_common(20))
print("**********************************************")
print("size of vocabulary after removing is",len(counter_uni))

top 20 frequency words after removing*******************************
[('neutral_corpus', 22333), ('positive_corpus', 19726), ('negative_corpus', 7767), ('get', 2464), ("it's", 2446), ('one', 2123), ('3rd', 1627), ('today', 1620), ('tonight', 1466), ('last', 1444), ("don't", 1435), ('make', 1389), ('love', 1264), ('come', 1258), ("can't", 1125), ('show', 1078), ('david', 1052), ('big', 1002), ('3', 994), ('happy', 987)]
**********************************************
size of vocabulary after removing is 5945


# Count the co-occurrence of any two tokens

In [22]:
count_collocation = util.collocation_counter(tweet_final,pmi_order)

In [23]:
print('the top 20 freqency collocation bigrams are **********************')
print(count_collocation.most_common(20))
print("**********************************************")
print('size of bi-gram counter is ',len(count_collocation))
print("**********************************************")

the top 20 freqency collocation bigrams are **********************
[(("it's", 'positive_corpus'), 1107), (('get', 'neutral_corpus'), 1052), (('love', 'positive_corpus'), 1038), (('get', 'positive_corpus'), 1009), (('one', 'positive_corpus'), 927), (("it's", 'neutral_corpus'), 920), (('happy', 'positive_corpus'), 886), (('one', 'neutral_corpus'), 870), (('today', 'positive_corpus'), 804), (('3rd', 'neutral_corpus'), 778), (('tonight', 'positive_corpus'), 777), (('great', 'positive_corpus'), 756), (('last', 'neutral_corpus'), 648), (('come', 'positive_corpus'), 642), (("can't", 'positive_corpus'), 635), (('wait', 'positive_corpus'), 632), (('last', 'positive_corpus'), 593), (('today', 'neutral_corpus'), 589), (('make', 'neutral_corpus'), 588), (('make', 'positive_corpus'), 579)]
**********************************************
size of bi-gram counter is  859930
**********************************************


In [24]:
## remove the low frequency bigrams
counter_col = util.remove_rare_term(count_collocation,rare_bigram)

In [25]:
print('size of bi-gram counter after removing rare bigrame is ',len(counter_col))
print("**********************************************")

size of bi-gram counter after removing rare bigrame is  37201
**********************************************


In [26]:
sorted_x = sorted(counter_col.items(), key=util.operator.itemgetter(1))
sorted_x.reverse()

In [27]:
print('the top 10 colloation words are*********************************')
print(sorted_x[:10])
print("**********************************************")

sorted_x.reverse()
print('the most low 10 colloation r words are*********************************')
print(sorted_x[:10])
print("**********************************************")

the top 10 colloation words are*********************************
[(("it's", 'positive_corpus'), 1107), (('get', 'neutral_corpus'), 1052), (('love', 'positive_corpus'), 1038), (('get', 'positive_corpus'), 1009), (('one', 'positive_corpus'), 927), (("it's", 'neutral_corpus'), 920), (('happy', 'positive_corpus'), 886), (('one', 'neutral_corpus'), 870), (('today', 'positive_corpus'), 804), (('3rd', 'neutral_corpus'), 778)]
**********************************************
the most low 10 colloation r words are*********************************
[(('yeah', 'guy'), 6), (('south', 'carolina'), 6), (("it's", 'safe'), 6), (('done', 'something'), 6), (('poland', 'positive_corpus'), 6), (('mikasa', 'neutral_corpus'), 6), (('search', 'negative_corpus'), 6), (('race', 'racing'), 6), (('gets', 'looks'), 6), (('found', 'news'), 6)]
**********************************************


# Construct word occurrences matrix and PMI matrix

In [28]:
# document = [['A', 'B'], ['C', 'B'], ['A', 'B', 'C', 'D']]
# counter_uni = util.word_counter(document)
# counter_col = util.collocation_counter(document)
df_coll,matrix_coll = util.construct_word_occurrence_matrix(counter_uni,counter_col)

In [29]:
print(df_coll)

              nirvana  demand  cape  howard  cst  tab  teacher  cougar  \
nirvana          10.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
demand            0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
cape              0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
howard            0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
cst               0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
tab               0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
teacher           0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
cougar            0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
ballot            0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
looking           0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
defeats           0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
movie             0.0     0.0   0.0     0.0  0.0  0.0      0.0     0.0   
oct               0.0     0.0   0.0   

In [30]:
names_index = df_coll.index.tolist()
names_header = df_coll.index.tolist()

In [31]:
dff_pmi,ppmi_matrix = util.PPMI_matrix(names_index,names_header,matrix_coll,la_smooth)

applying laplace smoothed, parameter is 1


In [32]:
print(dff_pmi)

               nirvana    demand      cape    howard       cst       tab  \
nirvana       3.228089  0.000000  0.000000  0.000000  0.000000  0.000000   
demand        0.000000  0.042609  0.026492  0.006536  0.044545  0.035370   
cape          0.000000  0.026492  0.010375  0.000000  0.028428  0.019253   
howard        0.000000  0.006536  0.000000  0.000000  0.008473  0.000000   
cst           0.000000  0.044545  0.028428  0.008473  0.046482  0.037306   
tab           0.000000  0.035370  0.019253  0.000000  0.037306  0.028131   
teacher       0.000000  0.029605  0.013488  0.000000  0.031541  0.022366   
cougar        0.000000  0.042850  0.026733  0.006778  0.044787  0.035612   
ballot        0.000000  0.045030  0.028913  0.008957  0.046966  0.037791   
looking       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
defeats       0.000000  0.045030  0.028913  0.008957  0.046966  0.037791   
movie         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
oct         

In [33]:
print('the highest PPMI value tuples is****************************')
print(dff_pmi.stack().index[util.np.argmax(ppmi_matrix)])
print()

the highest PPMI value tuples is****************************
('fighters', 'foo')



# Apply symmetrics SVD to get the word and context embedding matrix

In [34]:
W,C = util.construct_word_embedding(dimension,ppmi_matrix)

In [35]:
print(W)

[[-0.38012281  0.0250975   0.32396115 ...  0.06085082 -0.22318987
   0.00787766]
 [-0.01073546 -0.20584741 -0.00232953 ...  0.00118904  0.00048279
  -0.00088854]
 [-0.01961335 -0.13279468 -0.02828063 ...  0.00155697  0.02578281
  -0.1479699 ]
 ...
 [-0.02943224 -0.04553242 -0.00059477 ... -0.02400322 -0.0360831
  -0.0707499 ]
 [-0.29689284  0.00850647  0.27530284 ...  0.03548187  0.06587862
   0.05768785]
 [-0.0116884  -0.21934881 -0.00228434 ...  0.00311485  0.0014654
  -0.00528072]]


In [36]:
print('shape of W is**********************************************')
print(W.shape)
print("**********************************************")

print('shape of C is*********************************************')
print(C.shape)
print("**********************************************")

shape of W is**********************************************
(5945, 100)
**********************************************
shape of C is*********************************************
(5945, 100)
**********************************************


In [56]:
frame3 = frame2.div(frame2.sum(axis=1), axis=0)

In [37]:
header = dff_pmi.index.tolist()
frame=util.to_csv(W,header,'test_svd_embedding.csv')
print('export word embedding to csv file!!!')

export word embedding to csv file!!!


In [62]:
frame2 = frame.div(frame.sum(axis=1), axis=0)

In [65]:
frame2.iloc[0].sum()

1.0000000000000004

In [63]:
most_similar = util.show_most_similar('positive_corpus',frame2,20)

positive_corpus
it's
neutral_corpus
get
one
last
negative_corpus
come
great
3rd
don't
can't
3
really
show
wait
hope
concert
live
music
big
better
tuesday
free
4
october
lol
seeing
much
he's
july
tickets
u
playing
pi
amazing
david
awakens
home
please
november
sept
meet
little
join
look
guys
band
december
shirt
school
looking
team
fleetwood
tribute
thing
miss
starwars
wild
thriller
black
top
2015
vancouver
bad
merry
wanna
anyone
find
im
pll
sonic
tyler
horror
grade
cheese
made
remember
kind
test
proud
easter
listen
cara
real
painter
cinema
books
clarke
answer
cd
jackson's
bloody
polish
diego
seen
wichita
championship
stores
john


In [39]:
frame1 = frame.reset_index(drop=False)

label = []
for i in most_similar:
    label.append(frame1.iloc[i]['index'])

In [40]:
print(label)

['negative_corpus', 'neutral_corpus', "it's", 'get', 'one', 'positive_corpus', "don't", 'people', 'said', "he's", 'make', 'isis', 'eu', 'really', 'today', 'says', 'iraq', "that's", '3rd', 'last', 'u', "can't", 'lol', 'tories', 'elections', 'agreement', 'babies', 'shit', 'look', 'trump', 'mumbai', 'take', 'watching', 'chancellor', 'islam', 'sentence', 'horror', 'petition', 'yemen', 'coalition', 'german', 'islamic', 'fears', "blair's", 'ellen', 'muslims', 'spread', 'fatal', 'protect', "jenner's", 'wild', 'immigrants', 'santorum', 'village', 'women', 'staff', 'defeat', 'news', 'kurds', 'sect', 'bruce', 'buhari', 'better', 'soldiers', 'crash', 'im', 'yakub', 'hindus', 'sure', 'justice', 'borno', 'pray', 'troops', 'bad', 'controls', 'attack', 'mercy', "arabia's", "perry's", 'charge', 'accident', 'give', 'danny', "doesn't", 'nigerian', 'face', 'jews', 'much', 'army', 'love', 'sc', 'never', 'religious', 'pregnant', 'turkish', 'blasts', 'muslim', 'france', 'jail', 'december']


In [44]:
frame1.loc[0][1:].sum()

1.6547160609181961